In [145]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

In [142]:
champ2023_url = 'https://www.f1-world.ru/2023/komandy.php'

champ2023_page = requests.get(champ2023_url)
soup = BeautifulSoup(champ2023_page.text, "html.parser")
champ2023_info = soup.find('div', class_='midpart').find('table').findAll('a')

if champ2023_page.status_code == 200:
    print('Соединение успешно!')

Соединение успешно!


In [146]:
drivers_url = []
teams_url = []

for i in tqdm(champ2023_info):
    url = i.get('href')
    if 'teams' in url and url not in teams_url:
        teams_url.append(url)
    elif 'drivers' in url and url not in drivers_url:
        drivers_url.append(url) 

100%|██████████| 55/55 [00:00<?, ?it/s]


In [144]:
def get_driver_info(driver_url: str):

    stats = ['Участий в Гран-При', 
             'Подиумы', 
             'Старты в Гран-При', 
             ' Поул-позиции', 
             'Победы', 
             'Первые линии', 
             '2-е места', 
             'Лучшие круги', 
             '3-е места', 
             'Зачетные очки']

    page = requests.get(driver_url)
    soup = BeautifulSoup(page.text, "html.parser")

    info_raw = list(map(lambda x: x.text.strip(), soup.findAll('div', class_='news8')))[4:16]
    info = dict(zip(stats, list(map(float, info_raw[:-1]))))
    info['Команда'] = info_raw[-1]
    name = soup.find('h1').text.strip()

    return name, info

In [128]:
def get_team_info(driver_url: str):

    stats = ['Участий в Гран-При', 
             'Поул-позиции', 
             'Старты в Гран-При', 
             'Первые линии', 
             'Победы', 
             'Лучшие круги', 
             'Подиумы', 
             'Зачетные очки', 
             'Чемпионские титулы', 
             'Кубки конструкторов']

    page = requests.get(driver_url)
    soup = BeautifulSoup(page.text, "html.parser")

    info_raw = list(map(lambda x: x.text.strip(), soup.findAll('div', class_='news8')))[3:14]
    info = dict(zip(stats, list(map(float, info_raw))))
    name = soup.find('h1').text.strip()

    return name, info

In [148]:
teams_stats = [get_team_info(team) for team in teams_url]
teams_name = [info[0] for info in teams_stats]
stats = [info[1:] for info in teams_stats]

teams_df = pd.concat(pd.DataFrame(stat) for stat in stats)
teams_df.index = teams_name
teams_df

,Участий в Гран-При,Поул-позиции,Старты в Гран-При,Первые линии,Победы,Лучшие круги,Подиумы,Зачетные очки,Чемпионские титулы,Кубки конструкторов
Red Bull,367.0,93.0,366.0,174.0,110.0,94.0,260.0,7119.00,7.0,6.0
Ferrari,1075.0,248.0,1071.0,551.0,243.0,259.0,805.0,10516.80,15.0,16.0
Mercedes,290.0,138.0,290.0,259.0,125.0,105.0,288.0,7323.64,9.0,8.0
Alpine,63.0,2.0,63.0,1.0,1.0,0.0,4.0,433.00,8.0,4.0
McLaren,947.0,156.0,943.0,343.0,183.0,162.0,502.0,6563.50,12.0,8.0
Alfa Romeo,204.0,12.0,204.0,35.0,10.0,16.0,26.0,359.00,2.0,6.0
Aston Martin,69.0,2.0,68.0,5.0,2.0,1.0,8.0,368.00,12.0,7.0
Haas,163.0,4.0,163.0,0.0,4.0,2.0,0.0,249.00,9.0,5.0
AlphaTauri,80.0,2.0,80.0,1.0,1.0,2.0,2.0,300.00,9.0,6.0
Williams,832.0,128.0,812.0,273.0,114.0,133.0,313.0,3626.00,7.0,9.0


In [149]:
drivers_stats = [get_driver_info(driver) for driver in drivers_url]
drivers_name = [info[0] for info in drivers_stats]
stats = [info[1:] for info in drivers_stats]

drivers_df = pd.concat(pd.DataFrame(stat) for stat in stats)
drivers_df.index = drivers_name
drivers_df

,Участий в Гран-При,Подиумы,Старты в Гран-При,Поул-позиции,Победы,Первые линии,2-е места,Лучшие круги,3-е места,Зачетные очки,Команда
Макс Ферстаппен,182.0,95.0,182.0,30.0,51.0,55.0,28.0,29.0,16.0,2502.5,Red Bull Honda
Серхио Перес,257.0,34.0,254.0,3.0,6.0,10.0,14.0,11.0,14.0,1441.0,Red Bull Honda
Шарль Леклер,122.0,28.0,121.0,22.0,5.0,29.0,10.0,7.0,13.0,1034.0,Ferrari
Карлос Сайнс,181.0,18.0,180.0,5.0,2.0,11.0,5.0,3.0,10.0,965.5,Ferrari
Антонио Джовинацци,62.0,0.0,62.0,7.0,5.0,0.0,0.0,0.0,0.0,21.0,Alfa Romeo Ferrari
Джордж Расселл,101.0,10.0,101.0,2.0,1.0,9.0,2.0,6.0,7.0,445.0,Mercedes
Льюис Хэмилтон,329.0,197.0,329.0,104.0,103.0,175.0,57.0,65.0,38.0,4625.5,Mercedes
Мик Шумахер,44.0,0.0,43.0,6.0,6.0,0.0,0.0,0.0,0.0,12.0,Haas Ferrari
Эстебан Окон,130.0,3.0,130.0,3.0,1.0,0.0,1.0,0.0,1.0,409.0,Alpine Renault
Пьер Гасли,127.0,4.0,127.0,2.0,1.0,1.0,1.0,3.0,2.0,388.0,Alpine Renault
